In [1]:
import numpy as np
import torch
import torch.nn as nn
from PQC import getHaar
import matplotlib.pyplot as plt
from QuantumState import *
from getPQCvector import *
from PQC_Loss import *

In [2]:
class PQCNN(nn.Module):
    def __init__(self,qubits=2,num=1000,bins=250):
        super.__init__()
        self.fc1 = nn.Linear(6,6,bias=True)
        self.activation = nn.tanh()
        self.fc2 = nn.Linear(6,6,bias=True)
        self.qubits = qubits
        self.num = num
        self.bins = bins
        self.haar_hist = torch.from_numpy(getHaar(reps=self.num,bins=self.bins,qubits=self.qubits))
        self.criterion = nn.KLdivLoss(reduction='sum')
    def getPQCvector(self,theta):
        q = QuantumState(2)
        q.RX(0,theta[0].view(1,))
        q.RX(1,theta[1].view(1,))
        q.RZ(0,theta[2].view(1,))
        q.RZ(1,theta[3].view(1,))
        q.CRX(0,1,theta[4].view(1,))
        q.CRX(1,0,theta[5].view(1,))
        v = q.state.view(-1,1)
        return v
    def getPQCvectorparallel(self,num):
        input = torch.rand(6,num)
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        return x
    def loss(self,hist):
        return self.criterion(torch.log(hist),self.haar_hist)
    def train(self,epoch=100):
        for ep in range(epoch):
            Fidelity = None
            for i in self.num:
                v1 = getPQCvector(x)
                v2 = getPQCvector(x)
                F = torch.abs(torch.matmul(torch.conj(torch.transpose(v1,0,1)),v2))**2
                if Fidelity is None:
                    Fidelity = F.view(1,)
                else:
                    Fidelity = torch.cat((Fidelity,F.view(1,)),-1)
            hist = histgauss(Fidelity,bins=250,sigma=1/bins)



  Using cached torch-1.8.1-cp38-none-macosx_10_9_x86_64.whl (119.6 MB)


In [ ]:
num = 10
input = torch.rand(6,num)